Technical Report - Bike Share Data Analysis
---
## Sillhoute Score Analysis for optmizing KMeans

Diogo Pessoa
TODO - replace before submission
{$STUDENT_ID}


In [ ]:
features = ['start_station_id_index', 'end_station_id_index', 'day_period_index', 'week_day_index']

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=features, outputCol="features")
# # Scaling the features
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=False)

# # Combine the VectorAssembler and StandardScaler into a Pipeline
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml import Pipeline

# You can now define a pipeline that includes both the assembler and the scaler
pipeline = Pipeline(stages=[assembler, scaler])

# Fit and transform the DataFrame using the defined pipeline
sampled_df_scaled = pipeline.fit(sampled_df_with_added_features_indexed).transform(
    sampled_df_with_added_features_indexed)


In [ ]:
k_values = list(range(5, 16))

# Initialize an empty list to store silhouette scores
silhouette_scores = []

# Iterate over values of k

for k in k_values:
    # Initialize KMeans with the specified number of clusters (k) and a seed for reproducibility
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")

    # Fit the model to the data
    model = kmeans.fit(adf_kmeans)

    # Transform the dataset to include cluster predictions
    predictions = model.transform(adf_kmeans)

    # Evaluate the model
    silhouette = evaluator.evaluate(predictions)
    silhouette_scores.append(silhouette)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(k_values, silhouette_scores, 'bx-', color='orange')
plt.xlabel('k')
plt.ylabel('Silhouette score')
plt.title('Silhouette score vs. Number of clusters (k)')
plt.xticks(np.arange(min(k_values), max(k_values) + 1, 1.0))
plt.grid(True)
plt.savefig(os.path.join(images_path, 'silhouette_score_vs_number_of_clusters.png'))
plt.show()

# Output the optimal k based on silhouette score
optimal_k = k_values[silhouette_scores.index(max(silhouette_scores))]
print(f"The optimal number of clusters k is: {optimal_k}")
